# 1. Données équipes par match

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# uploader le kaggle.json préalablement récupéré
from google.colab import files
files.upload()

{}

In [ ]:
# Créer un dossier kaggle
! mkdir ~/.kaggle
# Copie le fichier kaggle.json dans le dossier kaggle
! cp kaggle.json ~/.kaggle/
# Change les permisions du fichier kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# données kaggle https://www.kaggle.com/shubhmamp/english-premier-league-match-data
! kaggle datasets download -d shubhmamp/english-premier-league-match-data

  0% 0.00/5.57M [00:00<?, ?B/s]
100% 5.57M/5.57M [00:00<00:00, 91.4MB/s]


In [ ]:
# creation d'un folder dataset (si celui ci existe déjà, changer le nom dataset ou ignorer cette cellule si ce sont les mêmes données)
# dans le drive, et extraction des données
! mkdir '/content/drive/My Drive/dataset'
! unzip english-premier-league-match-data.zip -d '/content/drive/My Drive/dataset'

mkdir: cannot create directory ‘/content/drive/My Drive/dataset’: File exists
Archive:  english-premier-league-match-data.zip
replace /content/drive/My Drive/dataset/datafile/season14-15/season_match_stats.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/My Drive/dataset/datafile/season14-15/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season14-15/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season15-16/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season15-16/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season16-17/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season16-17/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season17-18/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season17-18/season_stats.json  
  inflating: /content/drive/My Drive

In [3]:
import json
import pandas as pd
import numpy as np

In [4]:
team_stats_14_15_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season14-15/season_stats.json'))
team_stats_15_16_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season15-16/season_stats.json'))
team_stats_16_17_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season16-17/season_stats.json'))
team_stats_17_18_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season17-18/season_stats.json'))

match_stats_14_15_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season14-15/season_match_stats.json'))
match_stats_15_16_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season15-16/season_match_stats.json'))
match_stats_16_17_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season16-17/season_match_stats.json'))
match_stats_17_18_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season17-18/season_match_stats.json'))

In [5]:
def team_stats(json,teamloc):
  
  # les données de la team home seront disponibles à la posittion 0 et celles de la team away à la position 1
  # si l'argument teamloc n'est pas fourni, la fonction ne s'exécutera pas
  teamidx = 0 if teamloc == 'home' else 1 if teamloc == 'away' else "null"
  stats = pd.DataFrame()

  row=0

  # iteration dans les items du json
  for match_id, infos_match in json.items():
      stats.loc[row, 'match_id'] = match_id
      
      team = dict(list(infos_match.values())[teamidx])

      for column, team_info in team['team_details'].items():
          stats.loc[row, column] = team_info
      
      for column, team_stat in team['aggregate_stats'].items():
          stats.loc[row, column] = team_stat
      

      row += 1
  
  stats['date'] = pd.to_datetime(stats['date'], dayfirst=True)
  stats = stats.sort_values(by=['date', 'match_id'])
  stats = stats.reset_index(drop=True)
  
  stats['team_rating'] = stats['team_rating'].astype(float)
  
  for column in stats.columns[5:]:
      stats[column] = stats[column].astype(float)
  
  return stats


In [6]:
def match_stats(json):
    stats = pd.DataFrame()
    
    stats = pd.DataFrame.from_dict(json, orient='index')
    
    return stats

In [7]:
team_stats_home = {'season_14_15': team_stats(team_stats_14_15_json,'home').add_prefix('home_'), 
                   'season_15_16': team_stats(team_stats_15_16_json,'home').add_prefix('home_'), 
                   'season_16_17': team_stats(team_stats_16_17_json,'home').add_prefix('home_'),  
                   'season_17_18': team_stats(team_stats_17_18_json,'home').add_prefix('home_')}

team_stats_away = {'season_14_15': team_stats(team_stats_14_15_json,'away').add_prefix('away_'), 
                   'season_15_16': team_stats(team_stats_15_16_json,'away').add_prefix('away_'), 
                   'season_16_17': team_stats(team_stats_16_17_json,'away').add_prefix('away_'),  
                   'season_17_18': team_stats(team_stats_17_18_json,'away').add_prefix('away_')}

match_results = {'season_14_15': match_stats(match_stats_14_15_json), 
                 'season_15_16': match_stats(match_stats_15_16_json), 
                 'season_16_17': match_stats(match_stats_16_17_json),  
                 'season_17_18': match_stats(match_stats_17_18_json)}

In [8]:
# ajout de la saison des données home
team_stats_home['season_14_15']['season'] = '2014_2015'
team_stats_home['season_15_16']['season'] = '2015_2016'
team_stats_home['season_16_17']['season'] = '2016_2017'
team_stats_home['season_17_18']['season'] = '2017_2018'

In [9]:
# concatenation pour toutes les données home
df_home = pd.concat([team_stats_home['season_14_15'],team_stats_home['season_15_16'],team_stats_home['season_16_17'],team_stats_home['season_17_18']])

# concatenation pour toutes les données away
df_away = pd.concat([team_stats_away['season_14_15'],team_stats_away['season_15_16'],team_stats_away['season_16_17'],team_stats_away['season_17_18']])

# concatenation pour toutes les données match
df_match = pd.concat([match_results['season_14_15'],match_results['season_15_16'],match_results['season_16_17'],match_results['season_17_18']])

In [10]:
# concatenation away / home
df_merge = df_home.merge(df_away, left_on=['home_match_id'], right_on=['away_match_id'])

In [11]:
#df_merge.info()
#df_match.info()
#df_match.head()
df_merge.head()

,home_match_id,home_team_id,home_team_name,home_team_rating,home_date,home_att_goal_low_left,home_won_contest,home_possession_percentage,home_total_throws,home_att_miss_high_left,home_blocked_scoring_att,home_total_scoring_att,home_att_sv_low_left,home_total_tackle,home_att_miss_high_right,home_aerial_won,home_att_miss_right,home_att_sv_low_centre,home_aerial_lost,home_accurate_pass,home_total_pass,home_won_corners,home_shot_off_target,home_ontarget_scoring_att,home_goals,home_att_miss_left,home_fk_foul_lost,home_att_sv_low_right,home_att_goal_low_centre,home_att_sv_high_left,home_total_offside,home_att_goal_high_left,home_att_goal_low_right,home_att_miss_high,home_att_sv_high_centre,home_att_post_high,home_post_scoring_att,home_att_sv_high_right,home_att_pen_goal,home_att_post_right,...,away_date,away_won_corners,away_fk_foul_lost,away_won_contest,away_total_tackle,away_aerial_lost,away_possession_percentage,away_att_goal_low_left,away_total_pass,away_total_throws,away_total_offside,away_blocked_scoring_att,away_ontarget_scoring_att,away_aerial_won,away_accurate_pass,away_total_scoring_att,away_att_sv_low_left,away_goals,away_att_miss_high_right,away_att_miss_right,away_shot_off_target,away_att_miss_left,away_att_miss_high,away_att_goal_low_centre,away_att_goal_high_right,away_att_miss_high_left,away_att_sv_low_centre,away_att_goal_high_left,away_att_sv_high_centre,away_att_goal_low_right,away_att_sv_high_right,away_att_sv_high_left,away_penalty_save,away_att_sv_low_right,away_post_scoring_att,away_att_post_high,away_att_post_left,away_att_pen_goal,away_att_goal_high_centre,away_att_post_right
0,829513,13,Arsenal,7.015000,2014-08-16,1.0,12.0,76.0,21.0,1.0,3.0,14.0,2.0,26.0,1.0,23.0,1.0,1.0,17.0,640.0,730.0,9.0,5.0,6.0,2.0,2.0,13.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-08-16,3.0,19.0,7.0,33.0,23.0,24.0,1.0,222.0,18.0,1.0,2.0,2.0,17.0,127.0,4.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,829515,14,Leicester,6.714286,2014-08-16,NaN,6.0,36.7,12.0,NaN,3.0,11.0,NaN,13.0,NaN,27.0,2.0,NaN,14.0,265.0,344.0,3.0,5.0,3.0,2.0,1.0,16.0,1.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-08-16,6.0,10.0,9.0,19.0,27.0,63.3,NaN,605.0,17.0,NaN,5.0,3.0,14.0,509.0,13.0,NaN,2.0,NaN,1.0,5.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,829517,32,Manchester United,6.707143,2014-08-16,1.0,13.0,59.6,29.0,NaN,4.0,14.0,1.0,13.0,NaN,20.0,1.0,3.0,10.0,482.0,558.0,4.0,5.0,5.0,1.0,2.0,14.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,...,2014-08-16,NaN,20.0,4.0,19.0,20.0,40.4,NaN,383.0,23.0,1.0,1.0,4.0,10.0,307.0,5.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,829519,171,Queens Park Rangers,6.715000,2014-08-16,NaN,8.0,51.0,22.0,NaN,6.0,19.0,1.0,14.0,NaN,30.0,5.0,3.0,15.0,296.0,383.0,8.0,7.0,6.0,NaN,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,2014-08-16,9.0,10.0,4.0,17.0,30.0,49.0,1.0,382.0,26.0,2.0,4.0,4.0,15.0,289.0,11.0,NaN,1.0,2.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,829520,96,Stoke,6.799231,2014-08-16,NaN,9.0,63.1,36.0,NaN,6.0,12.0,2.0,27.0,NaN,30.0,1.0,NaN,9.0,432.0,517.0,2.0,4.0,2.0,NaN,2.0,14.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-08-16,8.0,9.0,4.0,19.0,30.0,36.9,1.0,289.0,35.0,1.0,2.0,1.0,9.0,197.0,7.0,NaN,1.0,NaN,NaN,4.0,1.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# récupération de l'heure dans les données match et fusion des df de_match & df_merge
df_match['date'] = df_match['date_string'].apply(lambda x: x.split(' ')[0])
df_match['date'] = pd.to_datetime(df_match['date'], dayfirst=True)
df_match['time'] = df_match['date_string'].apply(lambda x: x.split(' ')[1])
df_match = df_match[['home_team_id','away_team_id','date','time']]

df_merge = df_merge.rename(columns={"home_match_id": "match_id", "home_date": "date"})
df_merge = df_merge.merge(df_match, left_on = ['home_team_id','away_team_id','date'], right_on=['home_team_id','away_team_id','date'])

In [13]:
# les NaN dans le dataset sont équivalents à 0
#df_merge.info()
df_merge = df_merge.fillna(0)

In [14]:
# prep du réagencement des colonnes pour avoir les données générales: saison, date et heure au début
#df_merge.columns

In [15]:
# réagencement pour avoir les données d'ordre général au début et suppression de away_match_id et away_date qui sont redondantes
# suppression egalement de home_goals et away_goals qui seront redonantes avec dataset cotes
df_merge = df_merge[['match_id', 'season', 'date', 'time', 'home_team_id', 'home_team_name', 'home_team_rating',
        'home_att_goal_low_left', 'home_won_contest',
       'home_possession_percentage', 'home_total_throws',
       'home_att_miss_high_left', 'home_blocked_scoring_att',
       'home_total_scoring_att', 'home_att_sv_low_left', 'home_total_tackle',
       'home_att_miss_high_right', 'home_aerial_won', 'home_att_miss_right',
       'home_att_sv_low_centre', 'home_aerial_lost', 'home_accurate_pass',
       'home_total_pass', 'home_won_corners', 'home_shot_off_target',
       'home_ontarget_scoring_att', 'home_att_miss_left',
       'home_fk_foul_lost', 'home_att_sv_low_right',
       'home_att_goal_low_centre', 'home_att_sv_high_left',
       'home_total_offside', 'home_att_goal_high_left',
       'home_att_goal_low_right', 'home_att_miss_high',
       'home_att_sv_high_centre', 'home_att_post_high',
       'home_post_scoring_att', 'home_att_sv_high_right', 'home_att_pen_goal',
       'home_att_post_right', 'home_att_goal_high_right', 'home_att_post_left',
       'home_att_goal_high_centre', 'home_penalty_save', 
       'away_team_id', 'away_team_name', 'away_team_rating', 'away_won_corners', 'away_fk_foul_lost',
       'away_won_contest', 'away_total_tackle', 'away_aerial_lost',
       'away_possession_percentage', 'away_att_goal_low_left',
       'away_total_pass', 'away_total_throws', 'away_total_offside',
       'away_blocked_scoring_att', 'away_ontarget_scoring_att',
       'away_aerial_won', 'away_accurate_pass', 'away_total_scoring_att',
       'away_att_sv_low_left', 'away_att_miss_high_right',
       'away_att_miss_right', 'away_shot_off_target', 'away_att_miss_left',
       'away_att_miss_high', 'away_att_goal_low_centre',
       'away_att_goal_high_right', 'away_att_miss_high_left',
       'away_att_sv_low_centre', 'away_att_goal_high_left',
       'away_att_sv_high_centre', 'away_att_goal_low_right',
       'away_att_sv_high_right', 'away_att_sv_high_left', 'away_penalty_save',
       'away_att_sv_low_right', 'away_post_scoring_att', 'away_att_post_high',
       'away_att_post_left', 'away_att_pen_goal', 'away_att_goal_high_centre',
       'away_att_post_right']]

In [16]:
df_merge.head()

,match_id,season,date,time,home_team_id,home_team_name,home_team_rating,home_att_goal_low_left,home_won_contest,home_possession_percentage,home_total_throws,home_att_miss_high_left,home_blocked_scoring_att,home_total_scoring_att,home_att_sv_low_left,home_total_tackle,home_att_miss_high_right,home_aerial_won,home_att_miss_right,home_att_sv_low_centre,home_aerial_lost,home_accurate_pass,home_total_pass,home_won_corners,home_shot_off_target,home_ontarget_scoring_att,home_att_miss_left,home_fk_foul_lost,home_att_sv_low_right,home_att_goal_low_centre,home_att_sv_high_left,home_total_offside,home_att_goal_high_left,home_att_goal_low_right,home_att_miss_high,home_att_sv_high_centre,home_att_post_high,home_post_scoring_att,home_att_sv_high_right,home_att_pen_goal,...,away_team_name,away_team_rating,away_won_corners,away_fk_foul_lost,away_won_contest,away_total_tackle,away_aerial_lost,away_possession_percentage,away_att_goal_low_left,away_total_pass,away_total_throws,away_total_offside,away_blocked_scoring_att,away_ontarget_scoring_att,away_aerial_won,away_accurate_pass,away_total_scoring_att,away_att_sv_low_left,away_att_miss_high_right,away_att_miss_right,away_shot_off_target,away_att_miss_left,away_att_miss_high,away_att_goal_low_centre,away_att_goal_high_right,away_att_miss_high_left,away_att_sv_low_centre,away_att_goal_high_left,away_att_sv_high_centre,away_att_goal_low_right,away_att_sv_high_right,away_att_sv_high_left,away_penalty_save,away_att_sv_low_right,away_post_scoring_att,away_att_post_high,away_att_post_left,away_att_pen_goal,away_att_goal_high_centre,away_att_post_right
0,829513,2014_2015,2014-08-16,17:30:00,13,Arsenal,7.015000,1.0,12.0,76.0,21.0,1.0,3.0,14.0,2.0,26.0,1.0,23.0,1.0,1.0,17.0,640.0,730.0,9.0,5.0,6.0,2.0,13.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Crystal Palace,6.628571,3.0,19.0,7.0,33.0,23.0,24.0,1.0,222.0,18.0,1.0,2.0,2.0,17.0,127.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,829515,2014_2015,2014-08-16,15:00:00,14,Leicester,6.714286,0.0,6.0,36.7,12.0,0.0,3.0,11.0,0.0,13.0,0.0,27.0,2.0,0.0,14.0,265.0,344.0,3.0,5.0,3.0,1.0,16.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,Everton,6.610000,6.0,10.0,9.0,19.0,27.0,63.3,0.0,605.0,17.0,0.0,5.0,3.0,14.0,509.0,13.0,0.0,0.0,1.0,5.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,829517,2014_2015,2014-08-16,12:45:00,32,Manchester United,6.707143,1.0,13.0,59.6,29.0,0.0,4.0,14.0,1.0,13.0,0.0,20.0,1.0,3.0,10.0,482.0,558.0,4.0,5.0,5.0,2.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Swansea,6.886429,0.0,20.0,4.0,19.0,20.0,40.4,0.0,383.0,23.0,1.0,1.0,4.0,10.0,307.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,829519,2014_2015,2014-08-16,15:00:00,171,Queens Park Rangers,6.715000,0.0,8.0,51.0,22.0,0.0,6.0,19.0,1.0,14.0,0.0,30.0,5.0,3.0,15.0,296.0,383.0,8.0,7.0,6.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Hull,7.176429,9.0,10.0,4.0,17.0,30.0,49.0,1.0,382.0,26.0,2.0,4.0,4.0,15.0,289.0,11.0,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,829520,2014_2015,2014-08-16,15:00:00,96,Stoke,6.799231,0.0,9.0,63.1,36.0,0.0,6.0,12.0,2.0,27.0,0.0,30.0,1.0,0.0,9.0,432.0,517.0,2.0,4.0,2.0,2.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Aston Villa,6.767692,8.0,9.0,4.0,19.0,30.0,36.9,1.0,289.0,35.0,1.0,2.0,1.0,9.0,197.0,7.0,0.0,0.0,0.0,4.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Cotes par match

In [17]:
! pip install datapackage

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 72 kB 559 kB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 131 kB 45.1 MB/s 
     |████████████████████████████████| 126 kB 68.4 MB/s 
     |████████████████████████████████| 242 kB 75.5 MB/s 
     |████████████████████████████████| 8.0 MB 56.9 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 127 kB 61.7 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7399 sha256=b680a9939f6cac43517ce00f4338e0d80f7873156493c07af6d8df44136b7ebb
  Stored in directory: /root/.cache/pip/wheels/dd/0a/62/49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10765 sha256=ecad9bcca73f00b4a31e5fade644aba800f0dfb37f4bf7e24318ece30a680a0e
  Store

In [18]:
import datapackage

In [19]:
#utilisation du process indiqué sur le site des données

data_url = 'https://datahub.io/sports-data/english-premier-league/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources

cotes_1415 = pd.read_csv(resources[5].descriptor['path'])
cotes_1516 = pd.read_csv(resources[4].descriptor['path'])
cotes_1617 = pd.read_csv(resources[3].descriptor['path']) 
cotes_1718 = pd.read_csv(resources[2].descriptor['path']) 

In [20]:
# vérification du contenu des ressources
# resources[3].descriptor['path']

In [21]:
df_cotes = pd.concat([cotes_1415, cotes_1516, cotes_1617, cotes_1718])

In [22]:
#traitement de la date
df_cotes['date'] = df_cotes['Date'].apply(lambda x: pd.to_datetime(x, dayfirst=True))
df_cotes = df_cotes.drop(['Date'], axis=1)

In [23]:
# remplacement des noms des équipes qui ne sont pas les mêmes dans les deux datasets

old_names = sorted(df_cotes['HomeTeam'].unique())
new_names = sorted(df_merge['home_team_name'].unique())

#print(old_names)
#print(new_names)
df_cotes['HomeTeam'] = df_cotes['HomeTeam'].replace(old_names, new_names)
df_cotes['AwayTeam'] = df_cotes['AwayTeam'].replace(old_names, new_names)

In [24]:
# supprimer toutes les colonnes de cotes pour lesquelles il y a des valeur nulles et qu'on exploitera en tous cas pas dans une 1ère itération
df_cotes = df_cotes.dropna(axis='columns')

In [25]:
# supprimer les variables redondantes avec df_merge
# HS/AS = tirs, HST/AST = tirs cadrés (HC/AC = corners laissés finalement car ds le dataset stats c'est les "won corners" et donc peut-être différent)
# suppression également de la variable Div qui concerne la division car identique
df_cotes = df_cotes.drop(['Div','HS','AS','HST','AST'], axis=1)

In [26]:
df_cotes = df_cotes[['date', 'HomeTeam', 'AwayTeam', 'Referee', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'B365H', 'B365D',
       'B365A', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA',
       'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD',
       'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
       'PSCH', 'PSCD', 'PSCA']]

In [27]:
# export csv pour vérifier le résultat
# df_cotes.to_csv('df_cotes.csv')

# 3. Jointure données matches et cotes

In [28]:
df_stats_cotes = df_merge.merge(df_cotes, left_on = ['date', 'home_team_name', 'away_team_name'], right_on = ['date', 'HomeTeam', 'AwayTeam'])

In [29]:
df_stats_cotes = df_stats_cotes.drop(['HomeTeam','AwayTeam'], axis=1)

In [30]:
df_stats_cotes[['match_id','home_team_id','away_team_id']] =  df_stats_cotes[['match_id','home_team_id','away_team_id']].astype(int) 
#df_stats_cotes.info(verbose=True)

In [31]:
#reagencement pour avoir les autres infos d'ordre général au début (qui ne sont pas des features)
# on enleve time et referee
df_stats_cotes = df_stats_cotes[['match_id',
 'season',
 'date',
 'away_team_id',
 'away_team_name',
 'home_team_id',
 'home_team_name',
 'home_team_rating',
 'home_att_goal_low_left',
 'home_won_contest',
 'home_possession_percentage',
 'home_total_throws',
 'home_att_miss_high_left',
 'home_blocked_scoring_att',
 'home_total_scoring_att',
 'home_att_sv_low_left',
 'home_total_tackle',
 'home_att_miss_high_right',
 'home_aerial_won',
 'home_att_miss_right',
 'home_att_sv_low_centre',
 'home_aerial_lost',
 'home_accurate_pass',
 'home_total_pass',
 'home_won_corners',
 'home_shot_off_target',
 'home_ontarget_scoring_att',
 'home_att_miss_left',
 'home_fk_foul_lost',
 'home_att_sv_low_right',
 'home_att_goal_low_centre',
 'home_att_sv_high_left',
 'home_total_offside',
 'home_att_goal_high_left',
 'home_att_goal_low_right',
 'home_att_miss_high',
 'home_att_sv_high_centre',
 'home_att_post_high',
 'home_post_scoring_att',
 'home_att_sv_high_right',
 'home_att_pen_goal',
 'home_att_post_right',
 'home_att_goal_high_right',
 'home_att_post_left',
 'home_att_goal_high_centre',
 'home_penalty_save',
 'away_team_rating',
 'away_won_corners',
 'away_fk_foul_lost',
 'away_won_contest',
 'away_total_tackle',
 'away_aerial_lost',
 'away_possession_percentage',
 'away_att_goal_low_left',
 'away_total_pass',
 'away_total_throws',
 'away_total_offside',
 'away_blocked_scoring_att',
 'away_ontarget_scoring_att',
 'away_aerial_won',
 'away_accurate_pass',
 'away_total_scoring_att',
 'away_att_sv_low_left',
 'away_att_miss_high_right',
 'away_att_miss_right',
 'away_shot_off_target',
 'away_att_miss_left',
 'away_att_miss_high',
 'away_att_goal_low_centre',
 'away_att_goal_high_right',
 'away_att_miss_high_left',
 'away_att_sv_low_centre',
 'away_att_goal_high_left',
 'away_att_sv_high_centre',
 'away_att_goal_low_right',
 'away_att_sv_high_right',
 'away_att_sv_high_left',
 'away_penalty_save',
 'away_att_sv_low_right',
 'away_post_scoring_att',
 'away_att_post_high',
 'away_att_post_left',
 'away_att_pen_goal',
 'away_att_goal_high_centre',
 'away_att_post_right',
 'HF',
 'AF',
 'HY',
 'AY',
 'HR',
 'AR',
 'HTHG',
 'HTAG',
 'HTR',
 'FTHG',
 'FTAG',
 'FTR',
 'B365H',
 'B365D',
 'B365A',
 'LBH',
 'LBD',
 'LBA',
 'PSH',
 'PSD',
 'PSA',
 'WHH',
 'WHD',
 'WHA',
 'VCH',
 'VCD',
 'VCA',
 'Bb1X2',
 'BbMxH',
 'BbAvH',
 'BbMxD',
 'BbAvD',
 'BbMxA',
 'BbAvA',
 'BbOU',
 'BbMx>2.5',
 'BbAv>2.5',
 'BbMx<2.5',
 'BbAv<2.5',
 'BbAH',
 'BbAHh',
 'BbMxAHH',
 'BbAvAHH',
 'BbMxAHA',
 'BbAvAHA',
 'PSCH',
 'PSCD',
 'PSCA']]

In [32]:
#df_stats_cotes_bckp.info(verbose=True)
#type(df_stats_cotes['HTR'].loc[0])
#match_id                    object  - STR - à modifier   -> INT
#season                      object  - STR     
#time                        object  - STR    
#Referee                     object  - STR       
#home_team_id                object  - STR - à modifier   -> INT        
#home_team_name              object  - STR        
#away_team_id                object  - STR - à modifier   -> INT        
#away_team_name              object  - STR       
#FTR                         object  - STR        
#HTR                         object  - STR   
#df_stats_cotes['home_att_goal_low_left'].loc[0]     

In [33]:
# pour vérification des données création du fichier csv qui peut être téléchargé à partir de colab et se retrouver en cliquant sur l'icône folder à gauche
# df_stats_cotes.to_csv('df_stats_cotes.csv')

# 4. Pre-Processing



#### 1. Suppression variables inutiles

Après une première observation des données, les variables suivantes ne semblent pas pertinentes

* Supprimer att_goal_low_left / att_goal_low_centre / att_goal_high_left / att_goal_low_right / att_goal_high_right / att_goal_high_centre 
* Supprimer att_miss_high_left / att_miss_high_right / att_miss_right / att_miss_left / att_miss_high /  (tirs non cadres) 
* Supprimer att_sv_low_left / att_sv_low_centre / att_sv_low_right / att_sv_high_left / att_sv_high_centre / att_sv_high_right (tirs arretés) 
* Supprimer att_post_high / att_post_right / att_post_left
* Suppr away_fk_foul_lost, redondance avec HF / AF


In [34]:
df_stats_cotes = df_stats_cotes.drop(['home_att_goal_low_left','home_att_goal_low_centre','home_att_goal_high_left','home_att_goal_low_right','home_att_goal_high_right','home_att_goal_high_centre'],axis=1)

In [35]:
df_stats_cotes = df_stats_cotes.drop(['away_att_goal_low_left','away_att_goal_low_centre','away_att_goal_high_left','away_att_goal_low_right','away_att_goal_high_right','away_att_goal_high_centre'],axis=1)

In [36]:
df_stats_cotes = df_stats_cotes.drop(['home_att_miss_high_left','home_att_miss_high_right','home_att_miss_right','home_att_miss_left','home_att_miss_high'],axis=1)

In [37]:
df_stats_cotes = df_stats_cotes.drop(['away_att_miss_high_left','away_att_miss_high_right','away_att_miss_right','away_att_miss_left','away_att_miss_high'],axis=1)

In [38]:
df_stats_cotes = df_stats_cotes.drop(['home_att_sv_low_left','home_att_sv_low_centre','home_att_sv_low_right','home_att_sv_high_left','home_att_sv_high_centre','home_att_sv_high_right'],axis=1)

In [39]:
df_stats_cotes = df_stats_cotes.drop(['away_att_sv_low_left','away_att_sv_low_centre','away_att_sv_low_right','away_att_sv_high_left','away_att_sv_high_centre','away_att_sv_high_right'],axis=1)

In [40]:
df_stats_cotes = df_stats_cotes.drop(['home_att_post_high','home_att_post_right','home_att_post_left'],axis=1)

In [41]:
df_stats_cotes = df_stats_cotes.drop(['away_att_post_high','away_att_post_right','away_att_post_left'],axis=1)

In [42]:
df_stats_cotes = df_stats_cotes.drop(['home_fk_foul_lost','away_fk_foul_lost'],axis=1)

#### 2. Combiner accurate_pass / total_pass (Faire le rapport entre les deux)

In [43]:
df_stats_cotes['home_pass'] =  df_stats_cotes['home_accurate_pass'] / df_stats_cotes['home_total_pass']

In [44]:
df_stats_cotes['away_pass'] =  df_stats_cotes['away_accurate_pass'] / df_stats_cotes['away_total_pass']

#### 3. Suppression cotes non exploitables ou non pertinentes dans le cadre de l'analyse envisagée

In [45]:
# bases de calcul
df_stats_cotes = df_stats_cotes.drop(['Bb1X2','BbOU','BbAH'],axis=1)

In [46]:
# cotes min/max/moyenne/handicap asiatique/nb de goals
df_stats_cotes = df_stats_cotes.drop(['BbMxH','BbAvH','BbMxD','BbAvD','BbMxA','BbAvA','BbMx>2.5','BbAv>2.5','BbMx<2.5','BbAv<2.5','BbAHh','BbMxAHH','BbAvAHH','BbMxAHA','BbAvAHA'],axis=1)


#### 4. Ajout colonnes Moyenne des joueurs defense, milieux, attaque des equipes Home et Away

In [47]:
def players_stats(json, teamloc):
    
    IsAway = 0 if teamloc == 'home' else 1 if teamloc == 'away' else None

    stats = pd.DataFrame()
   
    row = 0
    for match_id, infos_match in json.items():
        
        home = dict(list(infos_match.values())[IsAway])
        for column, player_stat in home['Player_stats'].items():
            stats.loc[row, 'season'] = None
            stats.loc[row, 'match_id'] = match_id
            for column1, player_details in home['Player_stats'][column]['player_details'].items():
              stats.loc[row, column1] = player_details
            for column1, match_details in home['Player_stats'][column]['Match_stats'].items():
              stats.loc[row, column1] = match_details

            row+=1
    
    stats = stats.sort_values(by=['match_id'])
    stats = stats.reset_index(drop=True)

    for column in stats.columns[6:]:
        stats[column] = stats[column].astype(float)
    
    return stats

In [48]:
players_stats_home = {'season_14_15': players_stats(team_stats_14_15_json, 'home').add_prefix('home_'), 
                      'season_15_16': players_stats(team_stats_15_16_json, 'home').add_prefix('home_'), 
                      'season_16_17': players_stats(team_stats_16_17_json, 'home').add_prefix('home_'),  
                      'season_17_18': players_stats(team_stats_17_18_json, 'home').add_prefix('home_')}

players_stats_away = {'season_14_15': players_stats(team_stats_14_15_json, 'away').add_prefix('away_'), 
                      'season_15_16': players_stats(team_stats_15_16_json, 'away').add_prefix('away_'), 
                      'season_16_17': players_stats(team_stats_16_17_json, 'away').add_prefix('away_'),  
                      'season_17_18': players_stats(team_stats_17_18_json, 'away').add_prefix('away_')}

In [49]:
# concatenation pour toutes les données home
df_players_home = pd.concat([players_stats_home['season_14_15'], players_stats_home['season_15_16'], players_stats_home['season_16_17'], players_stats_home['season_17_18']])

# concatenation pour toutes les données away
df_players_away = pd.concat([players_stats_away['season_14_15'], players_stats_away['season_15_16'], players_stats_away['season_16_17'], players_stats_away['season_17_18']])

In [50]:
# on ne conserve que les joueurs qui ont une note
df_players_home_rated = df_players_home.loc[df_players_home['home_player_rating'] > 0,:]
df_players_away_rated = df_players_away.loc[df_players_away['away_player_rating'] > 0,:]

In [51]:
#df_players_away_rated[df_players_away_rated['match_id']== '959766'].head(20)
#df_players_away_rated.info()

In [52]:
# concatenation pour toutes les données home
df_players_home_rated = df_players_home_rated.rename(columns={"home_match_id": "match_id"})

# concatenation pour toutes les données away
df_players_away_rated = df_players_away_rated.rename(columns={"away_match_id": "match_id"})

In [53]:
# regroupement par poste
df_players_home_rated['home_player_position'] = df_players_home_rated['home_player_position_value']
df_players_home_rated['home_player_position'].replace(['1', '2', '3', '4', '5'], ['Goalkeeper', 'Defender', 'Midfielder', 'Forward', 'Substitute'], inplace = True)

df_players_away_rated['away_player_position'] = df_players_away_rated['away_player_position_value']
df_players_away_rated['away_player_position'].replace(['1', '2', '3', '4', '5'], ['Goalkeeper', 'Defender', 'Midfielder', 'Forward', 'Substitute'], inplace = True)

In [54]:
def position_rating(teamloc):
  
  output = pd.DataFrame()
  position_list = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward', 'Substitute']

  if teamloc == 'home':
    df = pd.DataFrame(df_players_home_rated.groupby(['match_id', teamloc + '_player_position']).mean()[teamloc + '_player_rating'].reset_index())
  elif teamloc == 'away':
    df = pd.DataFrame(df_players_away_rated.groupby(['match_id', teamloc + '_player_position']).mean()[teamloc + '_player_rating'].reset_index())
  
  df.index = df['match_id']

  for position in position_list:
    output = pd.concat([output, df.loc[df[teamloc + '_player_position'] == position,:]], axis = 1)
    output = output.rename(columns = {teamloc + '_player_rating':position.lower() + str('_') + teamloc + '_player_rating'})
    output = output.drop(['match_id', teamloc + '_player_position'], axis = 1)

  output = output.reset_index()
  output = output.rename(columns = {'index':'match_id'})

  return output

df_position_home = position_rating('home')

In [55]:
df_position_home_rating = position_rating('home')
df_position_away_rating = position_rating('away')

df_position_rating = df_position_home_rating.merge(df_position_away_rating, on = ['match_id'])
df_position_rating['match_id'] = df_position_rating['match_id'].astype(int)

df_stats_cotes = df_stats_cotes.merge(df_position_rating, on = ['match_id'])

In [56]:
df_position_away_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   match_id                       1520 non-null   object 
 1   goalkeeper_away_player_rating  1520 non-null   float64
 2   defender_away_player_rating    1520 non-null   float64
 3   midfielder_away_player_rating  1520 non-null   float64
 4   forward_away_player_rating     1519 non-null   float64
 5   substitute_away_player_rating  1516 non-null   float64
dtypes: float64(5), object(1)
memory usage: 71.4+ KB


In [57]:
df_stats_cotes.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1519
Data columns (total 85 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   match_id                       1520 non-null   int64         
 1   season                         1520 non-null   object        
 2   date                           1520 non-null   datetime64[ns]
 3   away_team_id                   1520 non-null   int64         
 4   away_team_name                 1520 non-null   object        
 5   home_team_id                   1520 non-null   int64         
 6   home_team_name                 1520 non-null   object        
 7   home_team_rating               1520 non-null   float64       
 8   home_won_contest               1520 non-null   float64       
 9   home_possession_percentage     1520 non-null   float64       
 10  home_total_throws              1520 non-null   float64       
 11  home_blocked_scor

In [58]:
# il y a certaines variables nulles, quelle stratégie faudra t il adapter?
df_stats_cotes = df_stats_cotes.drop(columns=['substitute_away_player_rating', 'substitute_home_player_rating']) 
df_stats_cotes['forward_away_player_rating'] = df_stats_cotes['forward_away_player_rating'].fillna(0)

In [59]:
df_stats_cotes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1519
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   match_id                       1520 non-null   int64         
 1   season                         1520 non-null   object        
 2   date                           1520 non-null   datetime64[ns]
 3   away_team_id                   1520 non-null   int64         
 4   away_team_name                 1520 non-null   object        
 5   home_team_id                   1520 non-null   int64         
 6   home_team_name                 1520 non-null   object        
 7   home_team_rating               1520 non-null   float64       
 8   home_won_contest               1520 non-null   float64       
 9   home_possession_percentage     1520 non-null   float64       
 10  home_total_throws              1520 non-null   float64       
 11  home_blocked_scor

#### 5. Récupération des données passées

In [60]:
# renommage du df pour simplifier ensuite
df = df_stats_cotes.copy()

In [61]:
# transformation de certaines données
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [62]:
# sélection d'un match
print(df.groupby(['season']).min()['match_id'])
print(df.groupby(['season']).max()['match_id'])

season
2014_2015     829513
2015_2016     958426
2016_2017    1080506
2017_2018    1190174
Name: match_id, dtype: int64
season
2014_2015     829892
2015_2016     959908
2016_2017    1080885
2017_2018    1190553
Name: match_id, dtype: int64


In [63]:
def previous_results(id, n):

  df_home_last_home_results = pd.DataFrame()
  df_home_last_results = pd.DataFrame()
  df_away_last_away_results = pd.DataFrame()
  df_away_last_results = pd.DataFrame()
  df_last_results = pd.DataFrame()

  # récupération du nom des équipes
  home = list(df[df['match_id'] == id]['home_team_name'])[0]
  away = list(df[df['match_id'] == id]['away_team_name'])[0]
  date = list(df[df['match_id'] == id]['date'])[0]

  #création des dataframes home
  df_home_last_home_results = df[(df['home_team_name'] == home) & (df['date'] < date)][:]
  df_home_last_results = df[((df['home_team_name'] == home) | (df['away_team_name'] == home)) & (df['date'] < date)][:]

  #création des dataframes away
  df_away_last_away_results = df[(df['away_team_name'] == away) & (df['date'] < date)][:]
  df_away_last_results = df[((df['home_team_name'] == away) | (df['away_team_name'] == away)) & (df['date'] < date)][:]

  #création des dataframes confrontation
  df_last_results = df[(((df['home_team_name'] == home) & (df['away_team_name'] == away)) | ((df['home_team_name'] == away) & (df['away_team_name'] == home))) & (df['date'] < date)][:]

  #selection des n derniers matchs (pour home_last_results et away_last_results seulement)
  df_home_last_home_results = df_home_last_home_results.sort_values(by = ['date'], ascending = False)
  df_home_last_results = df_home_last_results.sort_values(by = ['date'], ascending = False)
  df_away_last_away_results = df_away_last_away_results.sort_values(by = ['date'], ascending = False)
  df_away_last_results = df_away_last_results.sort_values(by = ['date'], ascending = False)
  df_last_results = df_last_results.sort_values(by = ['date'], ascending = False)

  if len(df_home_last_home_results) < n:
    df_home_last_home_results = df_home_last_home_results
  else:
    df_home_last_home_results = df_home_last_home_results[0:n]

  if len(df_home_last_results) < n:
    df_home_last_results = df_home_last_results
  else:
    df_home_last_results = df_home_last_results[0:n]

  if len(df_away_last_away_results) < n:
    df_away_last_away_results = df_away_last_away_results
  else:
    df_away_last_away_results = df_away_last_away_results[0:n]
  
  if len(df_away_last_results) < n:
    df_away_last_results = df_away_last_results
  else:
    df_away_last_results = df_away_last_results[0:n]
  
  if len(df_last_results) < n:
    df_last_results  =df_last_results
  else:
    df_last_results = df_last_results[0:n]

  return df_home_last_home_results, df_home_last_results, df_away_last_away_results, df_away_last_results, df_last_results

In [65]:
# 3 types de variables pour chaque match:
# - les variables d'ordre informatif qui ne sont pas pertinentes pour la modélisation: 
# 'match_id', 'season', 'date', 'time', 'Referee', 'home/away_team_id', 'home/away_team_name'

# - les variables potentiellement intéressantes comme features:
# 'home/away_team_rating', 'home/away_won_contest', 'home/away_possession_percentage', 'home_total_throws',
# 'home/away_blocked_scoring_att', 'home/away_total_scoring_att',
# 'home/away_total_tackle', 'home/away_aerial_won', 'home/away_aerial_lost',
# 'home/away_accurate_pass', 'home/away_total_pass', 'home/away_won_corners',
# 'home/away_shot_off_target', 'home/away_ontarget_scoring_att',
# 'home/away_total_offside', 'home/away_post_scoring_att', 'home/away_att_pen_goal',
# 'home/away_penalty_save', 'goalkeeper_home/away_player_rating', 'defender_home/away_player_rating',
# 'midfielder_home/away_player_rating', 'forward_home/away_player_rating'
# 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'FTR', 'HTHG', 'HTAG', 'HTR', 'home_pass', 'away_pass',

# - les variables de type target: résultat des matches et cotes
# 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A', 'LBH', 'LBD',
# 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA',
# 'PSCH', 'PSCD', 'PSCA'

# l'objectif ici est de remplacer les variables features par les moyennes des 3 matches précédents, en faisant la distinction
# home away, en partant de l'hypothèse que le fait d'être en home/away a un impact sur le résultat du match

In [174]:
# remise dans l'ordre des variables pour faciliter le retraitement étapes précédentes 
# (d'abord variables info, ensuite feats en distinguant home et away, 
# ensuite variables de type target), le résultat mi temps est supprimé car ne sera sans doute pas 
# utilisé, il faudrait peut être supprimer aussi les nombres de goals mi temps
df = df[['match_id', 'season', 'date', 'away_team_id', 'away_team_name',
       'home_team_id', 'home_team_name',
       # feats home
       'home_team_rating','home_won_contest', 'home_possession_percentage', 'home_total_throws',
       'home_blocked_scoring_att', 'home_total_scoring_att',
       'home_total_tackle', 'home_aerial_won', 'home_aerial_lost',
       'home_accurate_pass', 'home_total_pass', 'home_won_corners',
       'home_shot_off_target', 'home_ontarget_scoring_att',
       'home_total_offside', 'home_post_scoring_att', 'home_att_pen_goal',
       'home_penalty_save', 'HF', 'HY','HR', 'HTHG', 'home_pass', 'goalkeeper_home_player_rating',
       'defender_home_player_rating', 'midfielder_home_player_rating', 'forward_home_player_rating',
       # feats away
       'away_team_rating', 'away_won_contest', 'away_possession_percentage', 'away_total_throws',
       'away_blocked_scoring_att', 'away_total_scoring_att',
       'away_total_tackle', 'away_aerial_won', 'away_aerial_lost',
       'away_accurate_pass', 'away_total_pass', 'away_won_corners',  
        'away_shot_off_target', 'away_ontarget_scoring_att', 
       'away_total_offside', 'away_post_scoring_att', 'away_att_pen_goal',
        'away_penalty_save', 'AF',  'AY', 'AR', 'HTAG', 'away_pass', 'goalkeeper_away_player_rating',
       'defender_away_player_rating', 'midfielder_away_player_rating', 'forward_away_player_rating',
       # target
       'FTHG', 'FTAG', 'FTR', 'B365H',
       'B365D', 'B365A', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH',
       'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']]

In [175]:
# instanciation du df qui comportera données matches précédents avec récultat match actuel
df_results = pd.DataFrame(columns=df.columns, index=df.index.values.tolist())

In [176]:
#array qui regroupera les lignes non traitées pour cause de nb de matchs insuffisants pour faire les moyennes
index_non_traites = []

In [177]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1519
Data columns (total 82 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   match_id                       0 non-null      object
 1   season                         0 non-null      object
 2   date                           0 non-null      object
 3   away_team_id                   0 non-null      object
 4   away_team_name                 0 non-null      object
 5   home_team_id                   0 non-null      object
 6   home_team_name                 0 non-null      object
 7   home_team_rating               0 non-null      object
 8   home_won_contest               0 non-null      object
 9   home_possession_percentage     0 non-null      object
 10  home_total_throws              0 non-null      object
 11  home_blocked_scoring_att       0 non-null      object
 12  home_total_scoring_att         0 non-null      object
 13  hom

In [178]:
def add_prev_res(index,row,match_id,a,c):

  # on ne retient que les matches pour lesquels on a une visibilité sur les 3 précédents
  if (len(a) > 2 and len(c) > 2):
    
    results_home_mean = a[['home_team_rating','home_won_contest', 'home_possession_percentage', 'home_total_throws',
       'home_blocked_scoring_att', 'home_total_scoring_att',
       'home_total_tackle', 'home_aerial_won', 'home_aerial_lost',
       'home_accurate_pass', 'home_total_pass', 'home_won_corners',
       'home_shot_off_target', 'home_ontarget_scoring_att',
       'home_total_offside', 'home_post_scoring_att', 'home_att_pen_goal',
       'home_penalty_save', 'HF', 'HY','HR', 'HTHG', 'home_pass', 'goalkeeper_home_player_rating',
       'defender_home_player_rating', 'midfielder_home_player_rating', 'forward_home_player_rating']].mean()

    # récupération des features des 3 matches précédents away et calcul moyenne
    # il faudra ensuite rajouter ces moyennes à la place des variables de type features
    # dans le df_result nouvellement créé
   
    results_away_mean = c[['away_team_rating', 'away_won_contest', 'away_possession_percentage', 'away_total_throws',
       'away_blocked_scoring_att', 'away_total_scoring_att',
       'away_total_tackle', 'away_aerial_won', 'away_aerial_lost',
       'away_accurate_pass', 'away_total_pass', 'away_won_corners',  
        'away_shot_off_target', 'away_ontarget_scoring_att', 
       'away_total_offside', 'away_post_scoring_att', 'away_att_pen_goal',
        'away_penalty_save', 'AF',  'AY', 'AR', 'HTAG', 'away_pass', 'goalkeeper_away_player_rating',
       'defender_away_player_rating', 'midfielder_away_player_rating', 'forward_away_player_rating']].mean()
  
    # les variables de type informatif restent identiques, on les reporte ds df nouvellement créé
    # certaines lignes seront en nans (quand nb de matchs précédents non suffisants), mais les
    # index sont importants car les colonnes ne sont pas remplies en une seule fois, il faut donc
    # insérer au bon index, les lignes comportant des nans pourront par la suite être supprimées
    df_results.iloc[index,0:7] = row[0:7]

    # insertion des moyennes home
    df_results.iloc[index,7:34] = results_home_mean
    
    # insertion des moyennes away
    df_results.iloc[index,34:61] = results_away_mean

    # ajout des variables de type target a partir du df initial

    df_results.iloc[index,61:82] = row[61:82]

    # s'il n'y a pas de nb de matchs suffisant sur les matchs antérieurs pr
    # calculer les moyennes, afficher le match id pr info. Ces matchs ne seront pas inclus dans le df
  else:
      index_non_traites.append(index)
      print(index)
      print('Nb de matchs antérieurs non suffisant')
      print(f'Nb Résultats home: {len(a)}')
      print(f'Nb Résultats away: {len(c)}')

In [179]:
for index, row in df.iterrows():
  match_id = row['match_id']
  a, b, c, d, e = previous_results(match_id, 3)
  add_prev_res(index,row,match_id,a,c)

0
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
1
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
2
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
3
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
4
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
5
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
6
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
7
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
8
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
9
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
10
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
11
Nb de matchs antérieurs non suffisant
Nb Résultats home: 0
Nb Résultats away: 0
12
Nb de match

In [180]:
# nb de lignes non traitées
len(index_non_traites)

95

In [181]:
# suppression des lignes non traitées
to_drop = df_results.index[index_non_traites]
df_results.drop(to_drop, inplace=True)

In [183]:
df_results.head()

,match_id,season,date,away_team_id,away_team_name,home_team_id,home_team_name,home_team_rating,home_won_contest,home_possession_percentage,home_total_throws,home_blocked_scoring_att,home_total_scoring_att,home_total_tackle,home_aerial_won,home_aerial_lost,home_accurate_pass,home_total_pass,home_won_corners,home_shot_off_target,home_ontarget_scoring_att,home_total_offside,home_post_scoring_att,home_att_pen_goal,home_penalty_save,HF,HY,HR,HTHG,home_pass,goalkeeper_home_player_rating,defender_home_player_rating,midfielder_home_player_rating,forward_home_player_rating,away_team_rating,away_won_contest,away_possession_percentage,away_total_throws,away_blocked_scoring_att,away_total_scoring_att,...,away_aerial_lost,away_accurate_pass,away_total_pass,away_won_corners,away_shot_off_target,away_ontarget_scoring_att,away_total_offside,away_post_scoring_att,away_att_pen_goal,away_penalty_save,AF,AY,AR,HTAG,away_pass,goalkeeper_away_player_rating,defender_away_player_rating,midfielder_away_player_rating,forward_away_player_rating,FTHG,FTAG,FTR,B365H,B365D,B365A,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA
60,829595,2014_2015,2014-10-04 00:00:00,167,Manchester City,24,Aston Villa,6.51976,6.33333,38.1333,23.6667,2.33333,9,14,11.6667,15,281,370.333,5,4.66667,2,2,0.666667,0,0,10.3333,2,0,0.666667,0.756784,6.58667,6.55167,6.67089,6.925,7.16048,11.3333,56.7333,20,3.33333,16,...,14.6667,516.333,605.667,4.66667,6.33333,6.33333,0.666667,0.666667,0,0,10.6667,3.66667,0,1.33333,0.850091,6.69333,7.28,7.13433,7.94833,0,2,A,7.5,4.5,1.5,6.5,4.33,1.5,8.05,4.57,1.48,7,3.75,1.53,8,4.5,1.5,8.7,5,1.42
61,829605,2014_2015,2014-10-04 00:00:00,162,Crystal Palace,214,Hull,6.80214,8.33333,41.3,18,2,10,24.3333,18.3333,19,300.333,381.333,3.33333,3.66667,4.33333,0.333333,0.333333,0.333333,0,10.6667,1.33333,0.333333,1.33333,0.78393,6.10667,7.14778,6.943,7.08167,6.83214,9.33333,28.1,17.3333,2.66667,8.66667,...,19.6667,153.667,241.333,2.66667,2,4,1.33333,0,0.333333,0,17,2.33333,0.333333,1,0.634125,6.23333,6.93333,7.18483,7.18333,2,0,H,2.1,3.5,3.8,2.05,3.4,4,2.12,3.44,3.89,2.1,3.1,3.8,2.1,3.4,3.9,2.18,3.45,3.67
62,829610,2014_2015,2014-10-04 00:00:00,184,Burnley,14,Leicester,6.95762,8.33333,37.0667,19.6667,3,11.6667,18.6667,20,19.3333,263.333,346.667,3.33333,5,3.66667,2.33333,0,0.666667,0,13.6667,1.66667,0,1,0.759296,6.57667,7.0475,6.95,7.53167,6.45881,4,45.7,23.3333,4.33333,11,...,22,307,403.667,3.66667,4.66667,2,1.33333,0,0,0,10.6667,2,0,0,0.759945,6.11333,6.86167,6.4685,6.205,2,2,D,1.8,3.75,5,1.8,3.6,5,1.79,3.72,5.16,1.8,3.25,5,1.8,3.7,5,1.72,4.05,5.2
63,829615,2014_2015,2014-10-04 00:00:00,175,West Bromwich Albion,26,Liverpool,6.88846,9.33333,59.0667,20,6,18,18.3333,17.6667,10.6667,509,586.667,5.66667,7.33333,4.66667,2.33333,0.666667,0,0,8.33333,1.66667,0,0.333333,0.863823,6.52333,7.12167,7.09933,6.96,6.65933,5.33333,41.5667,16.6667,3.66667,11.3333,...,14,291,376.333,6.33333,5,2.66667,1.33333,0.333333,0,0,10,2,0,0,0.767269,6.54667,6.765,6.95033,6.14,2,1,H,1.44,5,7.5,1.45,4.75,7,1.46,4.78,7.95,1.44,4.33,7,1.45,4.8,8,1.48,4.69,7.7
64,829625,2014_2015,2014-10-04 00:00:00,96,Stoke,16,Sunderland,6.69738,11,44.0667,20,3.66667,11,20.6667,10.3333,10.3333,317.333,386,6.66667,5,2.33333,1.33333,0,0,0,11,2.33333,0,0.666667,0.82096,6.83667,6.65417,6.98089,6.63778,7.0043,8.66667,44.2,19,2.33333,11.3333,...,19.3333,291.333,372,3.33333,6,3,1.66667,0.333333,0,0,10,2.66667,0,0.333333,0.767497,6.64,7.355,7.07083,7.41833,3,1,H,2.55,3.25,3.1,2.45,3.2,3.1,2.55,3.23,3.14,2.6,3,2.9,2.55,3.12,3.2,2.67,3.25,2.96


In [184]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1425 entries, 60 to 1519
Data columns (total 82 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   match_id                       1425 non-null   object
 1   season                         1425 non-null   object
 2   date                           1425 non-null   object
 3   away_team_id                   1425 non-null   object
 4   away_team_name                 1425 non-null   object
 5   home_team_id                   1425 non-null   object
 6   home_team_name                 1425 non-null   object
 7   home_team_rating               1425 non-null   object
 8   home_won_contest               1425 non-null   object
 9   home_possession_percentage     1425 non-null   object
 10  home_total_throws              1425 non-null   object
 11  home_blocked_scoring_att       1425 non-null   object
 12  home_total_scoring_att         1425 non-null   object
 13  ho

In [185]:
df_results.to_csv('dataset_moyennes_matchs.csv')